<a href="https://colab.research.google.com/github/WZ151/ML-OF-WEIZHENG/blob/master/%E5%88%A9%E7%94%A8%E4%BA%8C%E6%AC%A1%E5%9E%8B%E4%BC%98%E5%8C%96%E8%A7%A3SVM%E5%AF%B9%E5%81%B6%E4%BC%98%E5%8C%96%E5%90%91%E9%87%8F%E5%BD%A2%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [205]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
import cvxopt
from cvxopt import matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
data = load_breast_cancer()##将数据集赋值给iris变量
#化为-1，1项
data.target[data.target==0] = -1

模型

In [213]:
class svmClass:
  def __init__(self,c):
    self.c=c
  def fill(self,x,y):
    self.X=np.array(x)
    self.Y=np.array(y)
    self.m = self.X.shape[0]
    self.n = self.X.shape[1]
  def train(self):
    m = self.X.shape[0]
    n = self.X.shape[1]
    #P
    P = np.multiply(self.Y @ self.Y.T , self.X @ self.X.T)
    #q
    q = np.ones((m,1))*-1
    #A
    A = self.Y.reshape((1,m))
    #b
    b = np.zeros((1,1))
    #G
    tmp1 = np.diag(np.ones(m))
    tmp2 = np.diag(np.ones(m)*(-1))
    G = np.vstack((tmp1,tmp2))
    #h
    tmp3 = np.ones(m)*(self.c)
    tmp4 = np.zeros(m)
    h = np.mat(np.hstack((tmp3,tmp4))).T
    P = matrix(P)
    q = matrix(q)
    G = matrix(G.astype('float'))
    h = matrix(h)
    A = matrix(A.astype('float'))
    b = matrix(b)
    sv = cvxopt.solvers.qp(P,q,G,h,A,b)
    w = np.zeros((n))
    for i in range(m):
      w = w + sv['x'][i]*self.Y[i]*self.X[i]
    self.w = w.reshape((n,1))
    alpha = np.array(sv['x'])
    TRUE = np.ones((1,n))
    TRUE = TRUE==1
    index = np.array(alpha>0.5).reshape((m,1))
    i_X = index @ TRUE
    tmp6 = self.X[i_X]
    num_of_s = len(tmp6)/n
    b = self.Y[alpha>0.5].T - tmp6.reshape((int(num_of_s),n)) @ w
    self.b = np.mean(b)
  def predict(self,x):
    b = np.ones((len(x),1))*self.b
    result = b + x @ self.w
    result[result>0] = 1
    result[result<0] = -1
    return result

参数准备

In [209]:
#设置参数
c = 1
Y = np.mat(data.target).T
X = data.data
train_X,test_X,train_y,test_y = train_test_split(X,Y,test_size=0.3,random_state=5)

In [214]:
#继承类
model = svmClass(c)
#添加
model.fill(train_X,train_y)
#训练
model.train()
#预测并比较准确率
print("#"*30,"accuracy_score","#"*30)
accuracy_score(model.predict(test_X),test_y)

     pcost       dcost       gap    pres   dres
 0: -9.9699e+01 -9.7008e+02  5e+03  3e+00  1e-08
 1: -6.5349e+01 -6.2221e+02  1e+03  5e-01  6e-09
 2: -4.9691e+01 -3.2586e+02  5e+02  2e-01  4e-09
 3: -3.9528e+01 -1.5149e+02  2e+02  7e-02  3e-09
 4: -3.5946e+01 -8.3584e+01  8e+01  2e-02  3e-09
 5: -3.6435e+01 -5.4455e+01  3e+01  8e-03  3e-09
 6: -3.6973e+01 -4.5732e+01  1e+01  3e-03  3e-09
 7: -3.7483e+01 -4.0983e+01  4e+00  2e-04  3e-09
 8: -3.7770e+01 -4.0003e+01  2e+00  1e-04  3e-09
 9: -3.7828e+01 -3.9957e+01  2e+00  9e-05  3e-09
10: -3.8239e+01 -3.9254e+01  1e+00  3e-05  3e-09
11: -3.8332e+01 -3.9071e+01  8e-01  1e-05  3e-09
12: -3.8609e+01 -3.8689e+01  8e-02  7e-07  3e-09
13: -3.8641e+01 -3.8644e+01  3e-03  3e-08  3e-09
14: -3.8642e+01 -3.8643e+01  3e-04  2e-09  4e-09
15: -3.8643e+01 -3.8643e+01  3e-06  2e-11  3e-09
Optimal solution found.
############################## accuracy_score ##############################


0.9824561403508771